# Generate PMTiles 

Take all the Parquet files, generate their FlatGeoBuff versions and generate regional PMTiles from those. Then merge all PMTiles to a single file for the final application.

In [1]:
import os
import tempfile
from pathlib import Path

import geopandas as gpd
from tqdm.auto import tqdm
from glob import glob

In [ ]:
pmtiles_destination = "/data/uscuni-ulce/pmtiles/"

In [4]:
# os.mkdir(pmtiles_destination)
os.mkdir(pmtiles_destination + "nuts1")

FileExistsError: [Errno 17] File exists: '/data/uscuni-ulce/pmtiles_v10/nuts1'

In [5]:
regions = glob('/data/uscuni-ulce/data_product/***.parquet')

In [6]:
regions = [r for r in regions if len(r) == 42]

## Buildings

In [7]:
folder = f"/data/uscuni-ulce/processed_data/clusters/{v}/"

In [8]:
for region in tqdm(regions):
    # Create a temporary directory
    with tempfile.TemporaryDirectory() as temp_dir:
        # Define the file path
        file_path = f"{temp_dir}/region.fgb"
        gpd.read_parquet(region).to_crs(4326).to_file(file_path)
        os.system(f"""pixi run tippecanoe -o {pmtiles_destination}nuts1/{Path(region).stem}.pmtiles -l buildings -pf -pk -A '<a href="https://uscuni.org/", target="_blank">Charles University</a>' -q --force {file_path}""")

  0%|          | 0/29 [00:00<?, ?it/s]

In [9]:
os.system(f"pixi run tile-join -o {pmtiles_destination}complete.pmtiles -f -pk {pmtiles_destination}nuts1/*.pmtiles")

14/9384/5162  

0

In [ ]:
folder = "/data/uscuni-ulce/grids/h3/"

In [ ]:
# os.mkdir(pmtiles_destination + "h3")

In [ ]:
for region in tqdm(regions.index[::-1]):
    # Create a temporary directory
    with tempfile.TemporaryDirectory() as temp_dir:
        # Define the file path
        file_path = f"{temp_dir}/region.fgb"
        gpd.read_parquet(f"{folder}{region}_clusters.pq").to_crs(4326).to_file(file_path)
        os.system(f"""pixi run tippecanoe -o {pmtiles_destination}h3/{region}.pmtiles -z 12 -l h3 --coalesce-densest-as-needed -A '<a href="https://uscuni.org/", target="_blank">Charles University</a>' -q --force {file_path}""")

In [ ]:
os.system(f"pixi run tile-join -o {pmtiles_destination}h3.pmtiles -f -pk {pmtiles_destination}h3/*.pmtiles")

In [ ]:
os.system("aws s3 --profile uscuni cp /data/uscuni-ulce/pmtiles/h3.pmtiles s3://uscuni-public")